In [16]:
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn import *


data = pd.read_csv('data/selected_features.csv') #path with data
data.head()

# Calculate the split point to split into train and test.
split_point = int(0.8 * len(data))

# Split the DataFrame
df_train = data.iloc[:split_point]
df_test = data.iloc[split_point:]

df_train.head()

,price actual,generation fossil hard coal,generation hydro pumped storage consumption,price actual ewm,generation fossil hard coal ewm,generation hydro pumped storage consumption ewm,generation other renewable,generation fossil oil ewm,generation other renewable ewm,generation other ewm
0,65.41,4821.0,863.0,65.410000,4821.000,863.0000,73.0,162.000,73.000,43.0
1,64.92,4755.0,920.0,65.165000,4788.000,891.5000,71.0,160.000,72.000,43.0
2,64.48,4581.0,1164.0,64.822500,4684.500,1027.7500,73.0,158.500,72.500,43.0
3,59.32,4131.0,1503.0,62.071250,4407.750,1265.3750,75.0,159.250,73.750,43.0
4,56.04,3840.0,1826.0,59.055625,4123.875,1545.6875,74.0,157.625,73.875,43.0


In [17]:

# drop the target value from the train data and use it as label

X_train = df_train.drop('price actual', axis =1)
y_train = df_train['price actual']

#drop time, bec tree cant handle the time label
# X_train = X_train.drop('', axis =1)

#same for test
X_test = df_test.drop('price actual', axis =1)
y_test = df_test['price actual']
# X_test = X_test.drop('', axis =1)


#train the stooopid tree
reg = xgb.XGBRegressor(n_estimators=10000)
reg.fit(X_train, y_train, verbose = False)

ImportError: sklearn needs to be installed in order to use this module

In [ ]:
#plot feature importance
xgb.plot_importance(reg)


In [2]:
#plotting of results. has to be adjusted.

print(reg.predict(X_test))


X_test['pv_measurements_Prediction'] = reg.predict(X_test)


data['price actual_Prediction'].plot(style='k', figsize=(10,5), label = 'train')
df_test['price actual'].plot(style='b', figsize=(10,5), label = 'test')
X_test['price actual_Prediction'].plot(style='r', figsize=(10,5), label = 'prediction')
plt.title('Pred')
plt.legend()

NameError: name 'reg' is not defined